# Data cleaning for IBM-HR

Non eliminare le righe, per favore!

In [108]:
# import libraries
import pandas as pd
import numpy as np

In [109]:
# concatenate two data sets
df1 = pd.read_csv('/home/damessina/Desktop/project_dataset/DatasetProject1/test.csv') #path file train
df2 = pd.read_csv('/home/damessina/Desktop/project_dataset/DatasetProject1/train.csv', header=0) #path file test
df = pd.concat([df1, df2], ignore_index=True)

In [110]:
# Number of observations
print(df.index.size)
# Number of attributes
print(df.columns.size)
# Check if indices were imposted right
df.tail()

1470
33


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1465,37.0,No,Travel_Rarely,1123,Research & Development,16,2,Medical,4,Female,...,2,80.0,2,1,3.0,2,17.0,0,0,0
1466,33.0,Yes,Non-Travel,249,Sales,6,4,Life Sciences,2,Female,...,2,80.0,0,12,2.0,2,11.0,7,7,7
1467,33.0,No,NaN,583,Sales,4,1,Marketing,3,Male,...,1,80.0,0,5,2.0,4,3.0,2,0,3
1468,45.0,No,Non-Travel,1174,Sales,3,4,Marketing,1,Male,...,4,80.0,0,15,2.0,3,10.0,4,0,1
1469,48.0,No,Travel_Frequently,1431,Research & Development,14,3,Medical,2,Female,...,3,80.0,0,0,NaN,1,22.0,0,0,0


In [111]:
df['Gender'].unique()

array(['Male', 'Female', nan], dtype=object)

## YearsAtCompany

In [112]:
# susbsitute missing values of YearsAtCompany with values of YearsInCurrentRole
for i,ii in enumerate(df['YearsAtCompany']):
    if np.isnan(ii)==True: df.replace(df['YearsAtCompany'][i],
                                      df['YearsInCurrentRole'][i],
                                      inplace=True)
# check null values
df['YearsAtCompany'].isnull().sum()

0

## MonthlyIncome

In [113]:
# mask that group values at YearsAtCompay by 5 | Divide all workers that are working 1-5 years, 5-10, 10-15 and so on
five_years = (df['YearsAtCompany'] // 5) + 1
# create a new column and fill the missing value of MonthlyIncome with median value of group
df["MonthIncomeFill"] = df.groupby(five_years)["MonthlyIncome"].transform(lambda x: x.fillna(x.median()))
# check that everything is fine, non-missing values should be same
df[['YearsAtCompany', 'MonthlyIncome', 'MonthIncomeFill']][0:40]

,YearsAtCompany,MonthlyIncome,MonthIncomeFill
0,2.0,4298.0,4298.0
1,8.0,5304.0,5304.0
2,13.0,6306.0,6306.0
3,3.0,6430.0,6430.0
4,5.0,2766.0,2766.0
5,20.0,4332.0,4332.0
6,20.0,5605.0,5605.0
7,5.0,4440.0,4440.0
8,3.0,8865.0,8865.0
9,1.0,3.0,3.0


In [114]:
# drop the column with missing values
df = df.drop('MonthlyIncome', 1)
# rename new column with old name
df = df.rename(columns={"MonthIncomeFill": "MonthlyIncome"})
# check null values
df['MonthlyIncome'].isnull().sum()

0

## Performance rating

In [115]:
# susbsitute missing values of PerformingRating with values 3 and 4 keeping the proportion
df['PerformanceRatingFill'] = df['PerformanceRating'].fillna(pd.Series(np.random.choice([3, 4], 
                                                      p=[0.85, 0.15], size=len(df.index))))

In [116]:
# drop the column PerformanceRating
df = df.drop('PerformanceRating', 1)
# rename new column with old name
df = df.rename(columns={"PerformanceRatingFill": "PerformanceRating"})
# check null values
df['PerformanceRating'].isnull().sum()

0

## TrainingTimesLastYear

In [117]:
#grouping column it is useful to discrete the variable YearsAtCompany
grouping=[]  

for row in df['YearsAtCompany']:
    if row <=5:
        grouping.append(5)
    elif row <=10:
        grouping.append(10)
    elif row <=15:
        grouping.append(15)
    elif row <=20:
        grouping.append(20)
    elif row <=25:
        grouping.append(25)
    else:
        grouping.append(30)

#convert a list into a Series and add it to the df
df['grouping']=grouping

In [118]:
# create a pivot table for analysing TrainingTimesLastYear in relation with JobRole and YearsAtCompany (grouped)
table = pd.pivot_table(df, values='TrainingTimesLastYear', columns='JobRole', 
                       index='grouping', aggfunc='mean')
# round the pivot table mean results
np.round(table)

JobRole,Healthcare Representative,Human Resources,Laboratory Technician,Manager,Manufacturing Director,Research Director,Research Scientist,Sales Executive,Sales Representative
grouping,,,,,,,,,
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
10,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
15,3.0,2.0,3.0,2.0,3.0,4.0,3.0,2.0,3.0
20,3.0,2.0,3.0,2.0,2.0,2.0,3.0,3.0,3.0
25,2.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0
30,3.0,NaN,3.0,3.0,4.0,3.0,4.0,2.0,NaN


In [119]:
allenamento = df['TrainingTimesLastYear']

# for loop to replace missing values with pivot table results
for i, row in enumerate(allenamento):
    if np.isnan(row)==True:
        if df['grouping'][i]==10 and df['JobRole'][i]=='Research Director': allenamento[i]=2
        elif df['grouping'][i]==15 and df['JobRole'][i]=='Sales Representative': allenamento[i]=2
        elif df['grouping'][i]==15 and df['JobRole'][i]=='Sales Executive': allenamento[i]=2
        elif df['grouping'][i]==15 and df['JobRole'][i]=='Research Director': allenamento[i]=4
        elif df['grouping'][i]==15 and df['JobRole'][i]=='Manager': allenamento[i]=2
        elif df['grouping'][i]==15 and df['JobRole'][i]=='Human Resurces': allenamento[i]=2
        elif df['grouping'][i]==20 and df['JobRole'][i]=='Manager': allenamento[i]=2
        elif df['grouping'][i]==20 and df['JobRole'][i]=='Human Resurces': allenamento[i]=2
        elif df['grouping'][i]==20 and df['JobRole'][i]=='Manufacturing Director': allenamento[i]=2
        elif df['grouping'][i]==25 and df['JobRole'][i]=='Healthcare Representative': allenamento[i]=2
        elif df['grouping'][i]==25 and df['JobRole'][i]=='Laboratory Technician': allenamento[i]=2
        elif df['grouping'][i]==25 and df['JobRole'][i]=='Research Scientist': allenamento[i]=2
        elif df['grouping'][i]==30 and df['JobRole'][i]=='Research Director': allenamento[i]=4
        elif df['grouping'][i]==30 and df['JobRole'][i]=='Research Scientist': allenamento[i]=4
        elif df['grouping'][i]==30 and df['JobRole'][i]=='Director': allenamento[i]=4
        else: allenamento[i]=3

In [120]:
df.drop(columns=['grouping'], inplace=True) #drop the 'grouping' column

In [121]:
df['TrainingTimesLastYear'].isnull().sum() #check missing values

0

## Age

In [122]:
#compute the difference between two attributes (start working Age)
diff = df['Age']-df['TotalWorkingYears']

In [123]:
#replace fake element with NaN values
for i, element in enumerate(diff):
    if element<18: 
        diff[i]=np.nan

In [124]:
#compute the mean of diff attribute and round it
mean=np.round(diff.mean())

In [125]:
#compute the hipotetical Age
newage =df['TotalWorkingYears']+mean

In [126]:
giorgio = df['Age']
#replace NaN values of 'Age' with the correspondent values of 'newage'
for i, val in enumerate(giorgio):
    if np.isnan(val)==True:
        giorgio[i]=newage[i]

In [127]:
#check for missing values
df['Age'].isnull().sum()

0

## Gender

In [128]:
#replace Male and Female with 0 and 1
df.replace('Male',0, inplace=True)
df.replace('Female',1, inplace=True)

In [129]:
#susbsitute missing values with values 0 and 1 keeping the proportion
df['Gender'] = df['Gender'].fillna(pd.Series(np.random.choice([0, 1], 
                                                              p=[0.6, 0.4], 
                                                              size=len(df.index))))

In [130]:
#check for missing values
df['Gender'].isnull().sum()

0

## BusinessTravel

In [131]:
# susbsitute missing values of BusinessTravel keeping the proportion
df['BusinessTravel'] = df['BusinessTravel'].fillna(pd.Series(np.random.choice(['Travel_Rarely', 'Travel_Frequently', 'Non-Travel'], 
                                                      p=[0.71, 0.19, 0.10], size=len(df))))
# check null values
df['BusinessTravel'].isnull().sum()

0

## Drop values of the variables with constant values

In [132]:
df.drop(columns=['Over18', 'StandardHours'], inplace=True)

## Export final csv

In [133]:
df.to_csv(path_or_buf='/home/damessina/Desktop/project_dataset/hr_cleaned.csv', index=False)